In [7]:
import numpy as np
import torch
import transformers
import re

In [2]:
data = np.load("data/clear_data.npy")

In [18]:
hidden_poem_data = []
for item in data:
    sens = re.split(r'[,.，。！？]', item)
    length = len(sens[0])
    hidden_word = [word[0] if word else '' for word in sens]
    hidden_poem_data.append(f'{length}{"".join(hidden_word)}->{item}')


In [19]:
np.save("data/hidden_poems.npy", hidden_poem_data)

['sadawwadawdawdaw', 'asdawwad', 'awdawdawd', 'awdawd', 'awd', 'awdawd', 'adawd', 'adawdaw']
